# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/trashcan/Programming/DataEngineeringNanodegree/DataModelingWithCassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1: 
> ### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### Table `session_song`: 
#### To get song details from our event log for a specified `session_id` and `item_in_session` we will need to include these columns in our `PRIMARY KEY`. It makes sense to use the `session_id` as the partition key as this will segregate events per session, and using item_in_session as a clustering column will allow us to use a WHERE clause in our query for the item_in_session. Each row will be uniquely identified by this `PRIMARY KEY` choice as well because the 2-tuple `session_id, item_in_session` should be unique.

In [8]:
query1 = """CREATE TABLE IF NOT EXISTS session_song 
            (session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))
"""

#### Create table

In [9]:
# Drop table to reset workspace
query1 = "DROP TABLE IF EXISTS session_song"

try:
    session.execute(query1)
except Exception as e:
    print(e)
    
# Create our table
query1 = """CREATE TABLE IF NOT EXISTS session_song 
            (session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))
"""

try:
    session.execute(query1)
except Exception as e:
    print(e)

#### Query 1 will return the `artist`, `song_title`, and `song_length` for a given `session_id` and `item_in_session` sorted by `item_in_session`.

In [10]:
## Query 1:  Give the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query1 = """SELECT artist, song_title, song_length
            FROM session_song
            WHERE session_id = 338 AND item_in_session = 4
"""

#### Insert data into `session_song`

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_song (artist, song_title, song_length, session_id, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Verify that the data has been inserted into the table

In [12]:
## SELECT statement to verify the data was entered into the table
query = """SELECT artist, song_title, song_length
           FROM session_song
           WHERE session_id = 338 AND item_in_session = 4
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    for k,v in row._asdict().items():
        print(f"{k}: {v}")
    print()

artist: Faithless
song_title: Music Matters (Mark Knight Dub)
song_length: 495.30731201171875



## Query 2:
> ### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### Table `session_user_song`:
#### To get song and user details from our event log for a specified `user_id` and `session_id` we will need to include these columns in our `PRIMARY KEY`. We will use `(user_id, session_id)` as a composite key for our partition key and `item_in_session` for our clustering column because we would like to include `user_id`, `session_id`, and `item_in_session` but only sort the results by `item_in_session`.  Each row will be uniquely identified by this `PRIMARY KEY` choice as well because the 3-tuple `user_id, session_id, item_in_session` should be unique.

In [13]:
query2 = """CREATE TABLE IF NOT EXISTS session_user_song 
            (user_id int, session_id int, item_in_session int, artist text, song_title text, first_name text, last_name text,
            PRIMARY KEY ((user_id, session_id), item_in_session))
"""

#### Create table

In [14]:
# Drop table to reset workspace
query2 = "DROP TABLE IF EXISTS session_user_song"
try:
    session.execute(query2)
except Exception as e:
    print(e)
    
# Create our table
query2 = """CREATE TABLE IF NOT EXISTS session_user_song 
            (user_id int, session_id int, item_in_session int, artist text, song_title text, first_name text, last_name text,
            PRIMARY KEY ((user_id, session_id), item_in_session))
"""

try:
    session.execute(query2)
except Exception as e:
    print(e)

#### Query 2 returns the `artist`, `song_title`, `first_name`, and `last_name` for a given `user_id` and `session_id` sorted by `item_in_session`.

In [15]:
## Query 2: Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query2 = """SELECT  artist, song_title, first_name, last_name
            FROM session_user_song
            WHERE user_id = 10 AND session_id = 182
"""                    

#### Insert data into `session_user_song`

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_user_song (artist, song_title, first_name, last_name, user_id, session_id, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

#### Verify that the data has been inserted into the table

In [17]:
# SELECT statement to verify the data was entered into the table
query = """SELECT  item_in_session, artist, song_title, first_name, last_name
            FROM session_user_song
            WHERE user_id = 10 AND session_id = 182
"""   

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    for k,v in row._asdict().items():
        print(f"{k}: {v}")
    print()

item_in_session: 0
artist: Down To The Bone
song_title: Keep On Keepin' On
first_name: Sylvie
last_name: Cruz

item_in_session: 1
artist: Three Drives
song_title: Greece 2000
first_name: Sylvie
last_name: Cruz

item_in_session: 2
artist: Sebastien Tellier
song_title: Kilometer
first_name: Sylvie
last_name: Cruz

item_in_session: 3
artist: Lonnie Gordon
song_title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)
first_name: Sylvie
last_name: Cruz



## Query 3:
> ### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Table `song_listens`:
#### To get `song_listens` by a user for a given `song_title` we will need to include `first_name`, `last_name`, and `song_title` in our `PRIMARY KEY`. We will use `song_title` and `user_id` for our `PRIMARY KEY` so that the data will be partitioned by `song_title` and we include `user_id` to ensure each row is unique.

In [18]:
query3 = """CREATE TABLE IF NOT EXISTS song_listens
            (song_title text, user_id int, first_name text, last_name text, PRIMARY KEY (song_title, user_id))
"""

#### Create table

In [19]:
# Drop table to reset our workspace
query3 = "DROP TABLE IF EXISTS song_listens"
try:
    session.execute(query3)
except Exception as e:
    print(e)
    
# Create our table
query3 = """CREATE TABLE IF NOT EXISTS song_listens
            (song_title text, user_id int, first_name text, last_name text, PRIMARY KEY (song_title, user_id))
"""

try:
    session.execute(query3)
except Exception as e:
    print(e)

#### Query 3 returns `first_name`, `last_name` for given `song_title`

In [20]:
## Query 3: Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query3 = """SELECT first_name, last_name
            FROM song_listens
            WHERE song_title = 'All Hands Against His Own'
"""                    

#### Insert data into `song_listens`

In [21]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listens (user_id, first_name, last_name, song_title)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

#### Verify that the data has been inserted into the table

In [22]:
query = """SELECT first_name, last_name, user_id
            FROM song_listens
            WHERE song_title = 'All Hands Against His Own'
""" 

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    for k,v in row._asdict().items():
        print(f"{k}: {v}")
    print()

first_name: Jacqueline
last_name: Lynch
user_id: 29

first_name: Tegan
last_name: Levine
user_id: 80

first_name: Sara
last_name: Johnson
user_id: 95



### Drop the tables before closing out the sessions

In [23]:
query1 = "DROP TABLE IF EXISTS session_song"
query2 = "DROP TABLE IF EXISTS session_user_song"
query3 = "DROP TABLE IF EXISTS song_listens"
try:
    session.execute(query1)
    session.execute(query2)
    session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()